In [1]:
include("multigrid.jl")
using Plots
using LinearAlgebra
using BenchmarkTools
nlist = [64, 128, 256, 512, 1024, 2048]


6-element Vector{Int64}:
   64
  128
  256
  512
 1024
 2048

# Run Experiment

In [2]:
errlist = zeros(length(nlist))
iterlist = zeros(length(nlist))
ct = 1
for n in nlist
    println("Begin computation for n = $n ... ")
    pde_er, u, v, p, iters = vcycle_stokes(n,3,3,1e-8,10,false)
    errlist[ct] = pde_er
    iterlist[ct] = iters
    println("n = $n, iters = $iters, error = $pde_er. Done.")
    ct += 1
end

Begin computation for n = 64 ... 


V-cycle iteration 1, residual: 0.020395395375584057
V-cycle iteration 2, residual: 0.000528887176354791
V-cycle iteration 3, residual: 2.0079694146637416e-5
V-cycle iteration 4, residual: 8.365365515161022e-7
V-cycle iteration 5, residual: 3.6730903732746403e-8
V-cycle iteration 6, residual: 1.6793573458300265e-9
n = 64, iters = 6, error = 0.0014950782818836106. Done.
Begin computation for n = 128 ... 
V-cycle iteration 1, residual: 0.020801565848039324
V-cycle iteration 2, residual: 0.0004677782422892084


V-cycle iteration 3, residual: 1.6433564387441977e-5
V-cycle iteration 4, residual: 7.038350285265232e-7
V-cycle iteration 5, residual: 3.1972058900128945e-8
V-cycle iteration 6, residual: 1.5159399843238678e-9
n = 128, iters = 6, error = 0.00037362812782173706. Done.
Begin computation for n = 256 ... 
V-cycle iteration 1, residual: 0.021015450598857576


V-cycle iteration 2, residual: 0.0004296238618766962
V-cycle iteration 3, residual: 1.2897564827215757e-5
V-cycle iteration 4, residual: 5.423598481140384e-7


V-cycle iteration 5, residual: 2.49212117593315e-8
V-cycle iteration 6, residual: 1.205157591843938e-9
n = 256, iters = 6, error = 9.339750802069553e-5. Done.
Begin computation for n = 512 ... 

...

V-cycle iteration 2, residual: 0.000408886315691082


V-cycle iteration 3, residual: 8.38229256609756e-6


V-cycle iteration 4, residual: 2.3813587544117918e-7


V-cycle iteration 5, residual: 9.952598063896643e-9


n = 2048, iters = 5, error = 1.432055045218045e-6. Done.


In [3]:
println(errlist)
println(iterlist)

[0.0014950782818836106, 0.00037362812782173706, 9.339750802069553e-5, 2.3348094292138517e-5, 5.836256896961707e-6, 1.432055045218045e-6]
[6.0, 6.0, 6.0, 6.0, 6.0, 5.0]


In [6]:
function empirical_coverge_order(errlist)
    # Check if the errlist has fewer than 2 elements
    if length(errlist) < 2
        error("errlist must contain at least two elements")
    end

    ratios = Float64[] # Initialize an empty array for ratios
    for i in 1:length(errlist)-1
        push!(ratios, errlist[i] / errlist[i+1])
    end

    rmean = mean(ratios) # Calculate mean of ratios
    log2_rmean = log2(rmean) # Calculate base-2 logarithm of the mean

    return log2_rmean
end

empirical_coverge_order (generic function with 1 method)

In [7]:
empirical_coverge_order(errlist)

2.0056234064018983

In [3]:
xticks = [64, 128, 256, 512, 1024, 2048]
ymin = errlist[1]
ymax = errlist[end]

scatter(nlist, errlist, label="error", xlabel="n", ylabel="error", title="error vs n (vcycle-dgs)", xscale=:log2, yscale=:log10, grid=true, markersize=4, xticks=xticks)
plot!(nlist, errlist, label="", xlabel="n", ylabel="error", xscale=:log2, yscale=:log10, grid=true, linewidth=2, xticks=xticks)
savefig("vcycle_dgs.pdf")

"d:\\numerical_linear_algebra\\final\\vcycle_dgs.pdf"

# Benchmark

## v1=3 v2=3

In [9]:
using BenchmarkTools
v1 = 3
v2 = 3
for n in nlist
    println("Benchmarking for n = $n...")
    @btime vcycle_stokes($n,v1,v2,1e-8,10,true)
    println("=====================================")
end

Benchmarking for n = 64...


  7.914 ms (3551 allocations: 19.42 MiB)
Benchmarking for n = 128...


  41.297 ms (4415 allocations: 77.49 MiB)
Benchmarking for n = 256...


  214.778 ms (5279 allocations: 310.42 MiB)
Benchmarking for n = 512...


  1.657 s (6144 allocations: 1.21 GiB)
Benchmarking for n = 1024...


  8.809 s (7008 allocations: 4.86 GiB)
Benchmarking for n = 2048...


  30.253 s (6566 allocations: 16.30 GiB)


In [5]:
for n in nlist
    x, xx, xxx , xxxx, iters = vcycle_stokes(n,3,3,1e-8,10,true)
    println("n = $n, iters = $iters")
end

n = 64, iters = 6


n = 128, iters = 6


n = 256, iters = 6


n = 512, iters = 6


n = 1024, iters = 6


n = 2048, iters = 5


## v1=2 v2=2

In [4]:
for n in nlist
    x, xx, xxx , xxxx, iters = vcycle_stokes(n,2,2,1e-8,10,true)
    println("n = $n, iters = $iters")
end

n = 64, iters = 7
n = 128, iters = 7


n = 256, iters = 7


n = 512, iters = 7


n = 1024, iters = 7


n = 2048, iters = 6


In [10]:
using BenchmarkTools
v1 = 2
v2 = 2
for n in nlist
    println("Benchmarking for n = $n...")
    @btime vcycle_stokes($n,v1,v2,1e-8,10,true)
    println("=====================================")
end

Benchmarking for n = 64...


  7.265 ms (3857 allocations: 20.28 MiB)
Benchmarking for n = 128...


  32.640 ms (4753 allocations: 80.81 MiB)
Benchmarking for n = 256...


  185.556 ms (5649 allocations: 323.61 MiB)
Benchmarking for n = 512...


  1.618 s (6546 allocations: 1.27 GiB)
Benchmarking for n = 1024...


  8.072 s (7442 allocations: 5.07 GiB)
Benchmarking for n = 2048...


  29.250 s (7152 allocations: 17.46 GiB)


## v1=4 v2=4

In [6]:
for n in nlist
    x, xx, xxx , xxxx, iters = vcycle_stokes(n,4,4,1e-8,10,true)
    println("n = $n, iters = $iters")
end

n = 64, iters = 5


n = 128, iters = 5


n = 256, iters = 5


n = 512, iters = 5


n = 1024, iters = 5


n = 2048, iters = 5


In [7]:
using BenchmarkTools
v1 = 4
v2 = 4
for n in nlist
    println("Benchmarking for n = $n...")
    @btime vcycle_stokes($n,v1,v2,1e-8,10,true)
    println("=====================================")
end

Benchmarking for n = 64...


  7.859 ms (3165 allocations: 17.91 MiB)
Benchmarking for n = 128...


  38.292 ms (3965 allocations: 71.53 MiB)
Benchmarking for n = 256...


  195.102 ms (4765 allocations: 286.62 MiB)
Benchmarking for n = 512...


  1.720 s (5566 allocations: 1.12 GiB)
Benchmarking for n = 1024...


  8.433 s (6366 allocations: 4.49 GiB)
Benchmarking for n = 2048...


  37.052 s (7166 allocations: 17.97 GiB)
